# Lab | BabyAGI with agent

**Change the planner objective below by changing the objective and the associated prompts and potential tools and agents - Wear your creativity and AI engineering hats
You can't get this wrong!**

You would need the OpenAI API KEY and the [SerpAPI KEY](https://serpapi.com/manage-api-keyhttps://serpapi.com/manage-api-key) to run this lab.


## BabyAGI with Tools

This notebook builds on top of [baby agi](baby_agi.html), but shows how you can swap out the execution chain. The previous execution chain was just an LLM which made stuff up. By swapping it out with an agent that has access to tools, we can hopefully get real reliable information

## Install and Import Required Modules

In [2]:
!pip install langchain_experimental

In [3]:
from typing import Optional

from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain_experimental.autonomous_agents import BabyAGI
from langchain_openai import OpenAI, OpenAIEmbeddings

## Connect to the Vector Store

Depending on what vectorstore you use, this step may look different.

In [8]:
!pip install faiss-cpu

   ---------------------------------------- 0.0/13.8 MB ? eta -:--:--
   ----- ---------------------------------- 1.8/13.8 MB 16.7 MB/s eta 0:00:01
   ---------------------------- ----------- 9.7/13.8 MB 30.2 MB/s eta 0:00:01
   ---------------------------------------  13.6/13.8 MB 31.7 MB/s eta 0:00:01
   ---------------------------------------- 13.8/13.8 MB 24.8 MB/s eta 0:00:00


In [9]:
!pip install google-search-results

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for google-search-results: filename=google_search_results-2.4.2-py3-none-any.whl size=32083 sha256=9b825c5417ae12ce06dcd4d7304b051a392e2e8d87e2461aec1b186124d05fa5
  Stored in directory: c:\users\larry\appdata\local\pip\cache\wheels\d3\b2\c3\03302d12bb44a2cdff3c9371f31b72c0c4e84b8d2285eeac53
Successfully built google-search-results


In [10]:
from langchain.docstore import InMemoryDocstore
from langchain_community.vectorstores import FAISS

In [11]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')
SERPAPI_API_KEY = os.getenv('SERPAPI_API_KEY')

In [61]:
# Define your embedding model
embeddings_model = OpenAIEmbeddings()
# Initialize the vectorstore as empty
import faiss

embedding_size = 1536
index = faiss.IndexFlatL2(embedding_size)
vectorstore = FAISS(embeddings_model.embed_query, index, InMemoryDocstore({}), {})

`embedding_function` is expected to be an Embeddings object, support for passing in a function will soon be removed.


## Define the Chains

BabyAGI relies on three LLM chains:
- Task creation chain to select new tasks to add to the list
- Task prioritization chain to re-prioritize tasks
- Execution Chain to execute the tasks


NOTE: in this notebook, the Execution chain will now be an agent.

In [62]:
from langchain.agents import AgentExecutor, Tool, ZeroShotAgent
from langchain.chains import LLMChain
from langchain_community.utilities import SerpAPIWrapper
from langchain_openai import OpenAI

todo_prompt = PromptTemplate.from_template(
    "You are a planner who is an expert at coming up with a todo list for a given objective. Come up with a todo list for this objective: {objective}"
)
todo_chain = LLMChain(llm=OpenAI(temperature=0), prompt=todo_prompt)
search = SerpAPIWrapper()
tools = [
    Tool(
        name="Search",
        func=search.run,
        description="useful for when you need to answer questions about current events",
    ),
    Tool(
        name="TODO",
        func=todo_chain.run,
        description="useful for when you need to come up with todo lists. Input: an objective to create a todo list for. Output: a todo list for that objective. Please be very clear what the objective is!",
    ),
]


prefix = """You are an AI who performs one task based on the following objective: {objective}. Take into account these previously completed tasks: {context}."""
suffix = """Question: {task}
{agent_scratchpad}"""
prompt = ZeroShotAgent.create_prompt(
    tools,
    prefix=prefix,
    suffix=suffix,
    input_variables=["objective", "task", "context", "agent_scratchpad"],
)

In [63]:
llm = OpenAI(temperature=0)
llm_chain = LLMChain(llm=llm, prompt=prompt)
tool_names = [tool.name for tool in tools]
agent = ZeroShotAgent(llm_chain=llm_chain, allowed_tools=tool_names)
agent_executor = AgentExecutor.from_agent_and_tools(
    agent=agent, tools=tools, verbose=True
)

### Run the BabyAGI

Now it's time to create the BabyAGI controller and watch it try to accomplish your objective.

In [64]:
OBJECTIVE = "Write a weather report for SF today"

In [65]:
# Logging of LLMChains
verbose = False
# If None, will keep on going forever
max_iterations: Optional[int] = 20
baby_agi = BabyAGI.from_llm(
    llm=llm,
    vectorstore=vectorstore,
    task_execution_chain=agent_executor,
    verbose=verbose,
    max_iterations=max_iterations,
)

In [66]:
response = baby_agi({"objective": OBJECTIVE})


*****TASK LIST*****

1: Make a todo list

*****NEXT TASK*****

1: Make a todo list


> Entering new AgentExecutor chain...
Thought: I should use the TODO action to create a todo list
Action: TODO
Action Input: "Create a todo list for writing a weather report for SF today"
Observation: 

1. Gather all necessary information: Start by collecting all the necessary information for writing a weather report for San Francisco today. This includes current temperature, humidity, wind speed, and any potential weather events.

2. Check reliable sources: Make sure to check reliable sources such as the National Weather Service or local news stations for accurate and up-to-date information.

3. Determine the format: Decide on the format for the weather report. Will it be a written report, a video, or a combination of both? This will help guide the rest of the process.

4. Create an outline: Before diving into writing, create an outline for the weather report. This will help organize the information 

In [67]:
response

{'objective': 'Write a weather report for SF today'}

In [70]:
# Retrieve all document IDs stored in the docstore
print("Document IDs stored in the docstore:")
print(vectorstore.docstore._dict.keys())  # Access the internal attribute "_dict"

Document IDs stored in the docstore:
dict_keys(['result_1_0', 'result_1_1', 'result_1_2', 'result_1_3', 'result_1_4', 'result_1_5', 'result_1_6', 'result_1_7', 'result_1_8', 'result_1_9', 'result_1_10', 'result_1_11', 'result_1_12', 'result_2_13', 'result_3_14', 'result_4_15', 'result_5_16', 'result_6_17', 'result_7_18', 'result_8_19'])


In [71]:

# Retrieve the content of a specific document by its ID
doc_id = "result_1_0"  # Change this to the ID you want to inspect
if doc_id in vectorstore.docstore._dict:
    print(f"Content for {doc_id}:")
    print(vectorstore.docstore._dict[doc_id])
else:
    print(f"Document ID {doc_id} not found in the docstore.")

Content for result_1_0:
page_content='The US-China trade war has had a significant impact on the global economy, with potential consequences for international trade and investment. The ongoing tensions and tariffs between the two countries have led to supply chain shifts and increased scrutiny of tech investments. The direct effects of the trade war on the global economy are limited, but there are potential long-term consequences if the tensions continue.' metadata={'task': 'Make a todo list'}


In [72]:

# Check the number of vectors stored in the FAISS index
print("Number of vectors in the FAISS index:")
print(vectorstore.index.ntotal)


Number of vectors in the FAISS index:
20


In [73]:
report = "BabyAGI Task Execution Report\n"
report += "=" * 30 + "\n\n"

for doc_id in vectorstore.docstore._dict.keys():
    report += f"Document ID: {doc_id}\n"
    report += f"Content: {vectorstore.docstore._dict[doc_id]}\n\n"  # Adjust if necessary

print(report)

BabyAGI Task Execution Report

Document ID: result_1_0
Content: page_content='The US-China trade war has had a significant impact on the global economy, with potential consequences for international trade and investment. The ongoing tensions and tariffs between the two countries have led to supply chain shifts and increased scrutiny of tech investments. The direct effects of the trade war on the global economy are limited, but there are potential long-term consequences if the tensions continue.' metadata={'task': 'Make a todo list'}

Document ID: result_1_1
Content: page_content='A todo list for understanding the impact of the US-China trade war on the global economy has been created.' metadata={'task': 'Research the current state of the US-China trade war and its impact on the global economy.'}

Document ID: result_1_2
Content: page_content='The todo list for assessing the impact of the US-China trade war on the global economy includes researching and gathering information, identifyin

In [74]:
# Generate a report and save it to a .txt file
report = "BabyAGI Task Execution Report\n"
report += "=" * 50 + "\n\n"

for task_id, task_content in vectorstore.docstore._dict.items():
    # Adjust based on actual structure of task_content
    report += f"Task ID: {task_id}\n"
    report += f"Content: {task_content.page_content}\n\n"

# Save the report to a .txt file
with open("babyagi_report.txt", "w") as f:
    f.write(report)

print("Report saved as 'babyagi_report.txt'")

Report saved as 'babyagi_report.txt'


In [75]:
# Empty the vectorstore
vectorstore.index.reset()  # Index FAISS reset
vectorstore.docstore = InMemoryDocstore({})  # Reboot the docstore